took from
[this link](http://kuanbutts.com/2017/08/08/how-to-pdna/)


In [49]:
# List of Imports
import osmnx as ox
import random
from shapely.geometry import Point, Polygon
from descartes import PolygonPatch
import pandas as pd
import pandana as pdna

In [50]:
# Bounding box for a small area in East Emeryville, South Berkeley and North Oakland

west, south, east, north = (-122.285535, 37.832531, -122.269571, 37.844596)

# Create a network from that bounding box

G = ox.graph_from_bbox(north, south, east, west, network_type='walk')

In [51]:
# Let's create n arbitrary points of interest (POI)

poi_count = 100

# this function makes those POI into Shapely points

def make_n_pois(north, south, east, west, poi_count):
    for poi in range(poi_count):
        x = (east - west) * random.random() + west
        y = (north - south) * random.random() + south
        yield Point(x, y)
        
pois = list(make_n_pois(north, south, east, west, poi_count))

In [52]:
# Create the plot fig and ax objects but prevent Matplotlib

# from plotting and closing out the plot operation

fig, ax = ox.plot_graph(G, fig_height=10, 
                        show=False, close=False,
                        edge_color='#777777')

# Instead, let's first update the network with these new random POI

for point in pois:
    patch = PolygonPatch(point.buffer(0.0001), fc='#ff0000', ec='k', linewidth=0, alpha=0.5, zorder=-1)
    ax.add_patch(patch)

In [62]:
G.edges

OutMultiEdgeView([(53022720, 93033416, 0), (53022720, 5483212937, 0), (53022720, 53090211, 0), (53086209, 53090197, 0), (53086209, 260582176, 0), (53086209, 240469589, 0), (53086209, 53086207, 0), (53086213, 53096366, 0), (53086213, 53096363, 0), (53086213, 53086217, 0), (53086213, 53085438, 0), (53086217, 53086219, 0), (53086217, 53113344, 0), (53086217, 53113341, 0), (53086217, 53086213, 0), (53086219, 53086217, 0), (53086219, 92951442, 0), (53086219, 53128526, 0), (1237076017, 2391023457, 0), (1237076017, 53129213, 0), (1237076017, 5483211593, 0), (1800890427, 1489746261, 0), (1800890427, 76210674, 0), (1800890427, 53126711, 0), (5483212874, 4918917078, 0), (1800890445, 53130589, 0), (1800890445, 53056102, 0), (1800890445, 1489303366, 0), (92951634, 53107940, 0), (92951634, 53085433, 0), (5483212886, 53090214, 0), (5483212886, 53045968, 0), (5483212886, 5483212888, 0), (5483212888, 5483212886, 0), (5483212889, 2817627471, 0), (5483212889, 2817627442, 0), (5483212889, 5483212891, 0),

In [53]:
# Given a graph, generate a dataframe (df)

# representing all graph nodes

def create_nodes_df(G):
    # first make a df from the nodes

    # and pivot the results so that the 

    # individual node ids are listed as

    # row indices

    nodes_df = pd.DataFrame(G.node).T
    
    # preserve these indices as a column values, too

    nodes_df['id'] = nodes_df.index
    # and cast it as an integer

    nodes_df['id'] = nodes_df['id'].astype(int)
    
    return nodes_df

nodes_df = create_nodes_df(G)

In [54]:
# Given a graph, generate a dataframe (df)

# representing all graph edges

def create_edges_df(G):
    # First, we must move the nested objects

    # to a signle top level dictionary

    # that can be consumed by a Pandas df

    edges_ref = {}
    
    # move through first key (origin node)

    for e1 in G.adj.keys():
        e1_dict = G.adj[e1]

        # and then get second key (destination node)

        for e2 in e1_dict.keys():
            # always use the first key here

            e2_dict = e1_dict[e2][0]

            # update the sub-dict to include

            # the origin and destination nodes

            e2_dict['st_node'] = e1
            e2_dict['en_node'] = e2

            # ugly, and unnecessary but might as

            # well name the index something useful

            name = '{}_{}'.format(e1, e2)

            # udpate the top level reference dict

            # with this new, prepared sub-dict

            edges_ref[name] = e2_dict

    # let's take the resulting dict and convert it

    # to a Pandas df, and pivot it as with the nodes

    # method to get unique edges as rows

    edges_df = pd.DataFrame(edges_ref).T
    
    # udpate the edge start and stop nodes as integers

    # which is necessary for Pandana

    edges_df['st_node'] = edges_df['st_node'].astype(int)
    edges_df['en_node'] = edges_df['en_node'].astype(int)
    
    # for the purposes of this example, we are not going

    # to both with impedence along edge so they all get

    # set to the same value of 1

    edges_df['weight'] = 1
    
    return edges_df

edges_df = create_edges_df(G)

In [56]:
nodes_df

,0,1,2,3,4,5,6,7,8,9,...,391,392,393,394,395,396,397,398,399,id
0,53022720,53086209,53086213,53086217,53086219,1237076017,1800890427,5483212874,1800890445,92951634,...,53128526,53033971,53129203,53022713,53022715,53086204,53129213,53129214,53086207,0


In [55]:
# Instantiate a Pandana (pdna) network (net)

net = pdna.Network(nodes_df['x'], nodes_df['y'],
                   edges_df['st_node'], edges_df['en_node'],
                   edges_df[['weight']])

KeyError: 'x'

In [ ]:
# Get the nearest node ids

near_ids = net.get_node_ids(pois_df['x'],
                            pois_df['y'],
                            mapping_distance=1)

# Set the response as a new column on the POI reference df

pois_df['nearest_node_id'] = near_ids

In [ ]:
# Create a merged dataframe that holds the node data (esp. x and y values)

# that relate to each nearest neighbor of each POI

nearest_to_pois = pd.merge(pois_df,
                           nodes_df,
                           left_on='nearest_node_id',
                           right_on='id',
                           how='left',
                           sort=False,
                           suffixes=['_from', '_to'])

In [ ]:
# Update the plot image with the nearest node on the graph

# highlighted and a line drawn from the node to the POI

for row_id, row in nearest_to_pois.iterrows():
    # Draw a circle on the nearest graph node

    point = Point(row.x_to, row.y_to)
    patch = PolygonPatch(point.buffer(0.0001),
                         fc='#0073ef',
                         ec='k',
                         linewidth=0,
                         alpha=0.5,
                         zorder=-1)
    ax.add_patch(patch)
    
    # Sloppy way to draw a line because I don't want to Google Matplotlib API 

    # stuff anymore right now

    linestr = LineString([(row['x_from'], row['y_from']),
                          (row['x_to'], row['y_to'])]).buffer(0.000001)
    new_line = PolygonPatch(linestr,
                            alpha=0.4,
                            fc='#b266ff',
                            zorder=1)
    ax.add_patch(new_line)

In [ ]:
npi = net.nearest_pois(1000,
                       'pois',
                       num_pois=5,
                       include_poi_ids=True)

In [ ]:
most_popular = npi.groupby('poi1').apply(lambda x: len(x)).to_frame().sort(0, ascending=False).head(1) # returns node id 93
